In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
## importing stuff
import os
import pickle
from sys import path as syspath
syspath.append(os.path.expanduser("~/srdjan_functs/"))
import javabridge
from bioformats import JARS as bfJARS
javabridge.start_vm(class_path=bfJARS, max_heap_size="20G")
import numpy as np
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

from IPython.display import display
import pandas as pd

from islets.Recording import Recording#, saveMovie
# from islets.Regions import Regions
# from islets.numeric import power_spectrum, rebin

from matplotlib.colors import LogNorm

In [36]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [3]:
from process_series import parse_leica

In [4]:
import parsing_1 as parser

In [5]:
parser.update_protocols()

In [65]:
protocols = parser.import_protocols(parseMetadata=False)

deleted  sex as not useful
deleted  frequency as not useful
deleted  resolution as not useful
when pH is NAN I will assume it is neutral (7.4).


In [66]:
protocols["recording"] = [x.split("_")[0] for x in protocols.experiment]
protocols["spreadsheet name"] = ["_".join(x.split("_")[1:]) for x in protocols.experiment]
protocols["Date"] = pd.to_datetime(protocols["date"], format="%Y_%m_%d", errors="coerce")

In [72]:
newProtocols = []

for (date,recname), df in protocols.groupby(["date","recording"]):
    if "Experiment54" not in recname: continue
    found = False
    for ext in ["lif","nd2"]:
        recfile = f"/data/Sandra/{date.split('_')[0]}/{date}/{recname}.{ext}"
        if os.path.isfile(recfile):
            found=True
            break
    if not found:
        print (f"{recname} is not where it is supposed to be ", f"/data/Sandra/{date.split('_')[0]}/{date}/")
        continue
    break
    newProtocols += [df.copy()]
    
    recmeta = f"/data/Sandra/{date.split('_')[0]}/{date}/.{recname}.{ext}.meta"

In [73]:
rec = Recording(recfile)
rec.calc_gaps()
indices = parse_leica(rec, index=True)

for ii, outname in indices:
    if len(ii)>1:
        row = rec.metadata.loc[ii[0]].copy()
        row["Name"] = outname
        row["End time"] = rec.metadata.loc[ii[-1],"End time"]
        row["compound"] = rec.metadata.loc[ii,["Name","Start time","End time","Duration","gap"]].copy()
        rec.metadata.drop(index=ii, inplace=True)
        rec.metadata = rec.metadata.append(row)
rec.metadata["Duration"] = rec.metadata["End time"] - rec.metadata["Start time"]
rec.metadata.sort_values("Name", inplace=True)

merged = pd.merge(rec.metadata,df,
         how="outer", right_on="spreadsheet name",left_on="Name")

# merged["series"] = merged["Name"].copy()

merged["Date"] = df.Date.iloc[0]
merged["recording"] = df.recording.iloc[0]

for k in ["date","experiment"]:
    del merged[k]

In [74]:
firstOrder = ["Date","recording","spreadsheet name","Name","Slice number"]
firstOrder += ['Frequency', 'Duration','SizeT', 'SizeX', 'SizeY', 'pxSize', 'pH', 'glucose', 'treatment', 'comments', 'microscope', 'dye',]
merged = merged[firstOrder+[c for c in merged.columns if c not in firstOrder]]

In [75]:
merged

,Date,recording,spreadsheet name,Name,Slice number,Frequency,Duration,SizeT,SizeX,SizeY,pxSize,pH,glucose,treatment,comments,microscope,dye,pxUnit,bit depth,Start time,End time,gap,mouse/human,part of pancreas,pH_,glucose_,magnification
0,2020-06-18,Experiment54a,Series003,Series003,1.0,9.89,00:12:26.807124,7385,256,256,0.857,7.4,15.0,nan,"31yr old female, BMI = 20.3, HbA1c = 4.8. Prep...",leica CF_SP8,Calbryte-520,µm,uint8,2020-06-18 13:43:48,2020-06-18 13:56:14.807124239,NaN,human,isolated islets,7.4,15,20x
1,2020-06-18,Experiment54a,Series004,Series004,2.0,19.80,00:23:42.698572,28132,128,128,1.530,7.4,8.0,nan,very long events,leica CF_SP8,Calbryte-520,µm,uint8,2020-06-18 14:09:14,2020-06-18 14:32:56.698572968,779.192876,BL6J,NaN,7.4,8,20x


In [61]:
Recording(recfile).metadata

,Name,SizeT,SizeX,SizeY,pxSize,pxUnit,bit depth,Frequency,Start time,End time,Duration
0,Image006,1,1024,1024,0.243,µm,uint8,NaN,2019-08-27 09:44:57,NaT,NaT
1,Image010,1,1024,1024,0.324,µm,uint8,NaN,2019-08-27 09:49:28,NaT,NaT
2,Image012,1,1024,1024,0.324,µm,uint8,NaN,2019-08-27 09:50:10,NaT,NaT
3,Series013,7583,512,512,0.648,µm,uint8,1.980,2019-08-27 09:50:40,2019-08-27 10:54:27.360729996,01:03:47.360729
4,Image017,1,1024,1024,0.241,µm,uint8,NaN,2048-03-12 10:41:23,NaT,NaT
5,Image018,1,1024,1024,0.241,µm,uint8,NaN,2019-08-27 11:04:23,NaT,NaT
6,Series019,2037,512,512,0.482,µm,uint8,1.980,2019-08-27 11:05:13,2019-08-27 11:22:20.631483619,00:17:07.631483
7,Image021,1,1024,1024,0.424,µm,uint8,NaN,2019-08-27 11:25:48,NaT,NaT
8,Series022,1,1024,1024,0.424,µm,uint8,NaN,2048-03-12 11:03:52,NaT,NaT
9,Series023,984,512,512,0.850,µm,uint8,1.980,2019-08-27 11:26:47,2019-08-27 11:35:03.318387781,00:08:16.318387


In [44]:
newProtocols = pd.concat(newProtocols,ignore_index=True)

In [45]:
newProtocols

,Date,recording,series,Slice number,date,experiment,microscope,mouse/human,pH,glucose,treatment,comments,dye,part of pancreas,pH_,glucose_,magnification
0,2019-06-10,Experiment21,Series021,1.0,2019_06_10,Experiment21_Series021,Leica upright,NMRI,7.4,12.0,pH changes (7.4 63-1240; 7.1 1240-1856; 7.7 18...,NaN,Calbryte-520,NaN,NaN,12,20x
1,2019-06-10,Experiment21,Series029,2.0,2019_06_10,Experiment21_Series029,Leica upright,NMRI,7.4,8.0,pH changes (7.4 42-1286; 7.1 1286-1920; 7.7 19...,NaN,Calbryte-520,NaN,NaN,8,20x
2,2019-06-10,Experiment21,Series034,3.0,2019_06_10,Experiment21_Series034,Leica upright,NMRI,7.4,8.0,pH changes (7.4 36-1286; 7.1 1286-1921; 7.7 19...,NaN,Calbryte-520,NaN,NaN,8,20x
3,2019-08-13,Experiment29,Series052-060,NaN,2019_08_13,Experiment29_Series052-060,Leica upright,NMRI,7.4,6.0,100 nM ryanodine,NaN,Calbryte-520,NaN,7.4,6,20x
4,2019-08-13,Experiment29,Series061,NaN,2019_08_13,Experiment29_Series061,Leica upright,NMRI,7.4,6.0,100 nM ryanodine,NaN,Calbryte-520,NaN,7.4,6,20x
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,2020-07-01,Experiment58c,Series002,3.0,2020_07_01,Experiment58c_Series002,leica CF_SP8,BL6J,7.4,8.0,nan,two islets,Calbryte-520,NaN,7.4,8,20x
341,2020-07-01,Experiment58d,Series001,4.0,2020_07_01,Experiment58d_Series001,leica CF_SP8,BL6J,7.4,8.0,nan,NaN,Calbryte-520,NaN,7.4,8,20x
342,2020-07-01,Experiment58d,Series003,5.0,2020_07_01,Experiment58d_Series003,leica CF_SP8,BL6J,7.4,8.0,nan,NaN,Calbryte-520,NaN,7.4,8,20x
343,2020-07-01,Experiment58d,Series006,6.0,2020_07_01,Experiment58d_Series006,leica CF_SP8,BL6J,7.4,8.0,nan,NaN,Calbryte-520,NaN,7.4,8,20x


In [39]:
if not os.path.isfile(recmeta):
    print ()

True

In [29]:
firstOrder = ["Date","recording","series","Slice number"]
protocols = protocols[firstOrder+[c for c in protocols.columns if c not in firstOrder]]
protocols.sort_values(firstOrder, inplace=True)

In [30]:
protocols

,Date,recording,series,Slice number,date,experiment,microscope,mouse/human,pH,glucose,treatment,comments,dye,part of pancreas,pH_,glucose_,magnification
0,2019-06-10,Experiment21,Series021,1.0,2019_06_10,Experiment21_Series021,Leica upright,NMRI,7.4,12.0,pH changes (7.4 63-1240; 7.1 1240-1856; 7.7 18...,NaN,Calbryte-520,NaN,NaN,12,20x
1,2019-06-10,Experiment21,Series029,2.0,2019_06_10,Experiment21_Series029,Leica upright,NMRI,7.4,8.0,pH changes (7.4 42-1286; 7.1 1286-1920; 7.7 19...,NaN,Calbryte-520,NaN,NaN,8,20x
2,2019-06-10,Experiment21,Series034,3.0,2019_06_10,Experiment21_Series034,Leica upright,NMRI,7.4,8.0,pH changes (7.4 36-1286; 7.1 1286-1921; 7.7 19...,NaN,Calbryte-520,NaN,NaN,8,20x
3,2019-07-01,Experimen22,Series011,1.0,2019_07_01,Experimen22_Series011,Leica upright,NMRI,7.4,12.0,pH changes (7.4 41-1278; 7.1 1278-1944; 7.7 19...,old mouse 14 Months,Calbryte-520,NaN,NaN,12,20x
4,2019-07-01,Experimen22,Series025,2.0,2019_07_01,Experimen22_Series025,Leica upright,NMRI,7.4,8.0,pH changes (7.4 71-1297; 7.1 1297-1959; 7.7 19...,old mouse 14 Months,Calbryte-520,NaN,NaN,8,20x
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538,2020-07-01,Experiment58c,Series002,3.0,2020_07_01,Experiment58c_Series002,leica CF_SP8,BL6J,7.4,8.0,nan,two islets,Calbryte-520,NaN,7.4,8,20x
539,2020-07-01,Experiment58d,Series001,4.0,2020_07_01,Experiment58d_Series001,leica CF_SP8,BL6J,7.4,8.0,nan,NaN,Calbryte-520,NaN,7.4,8,20x
540,2020-07-01,Experiment58d,Series003,5.0,2020_07_01,Experiment58d_Series003,leica CF_SP8,BL6J,7.4,8.0,nan,NaN,Calbryte-520,NaN,7.4,8,20x
531,2020-07-01,Experiment58d,Series006,6.0,2020_07_01,Experiment58d_Series006,leica CF_SP8,BL6J,7.4,8.0,nan,NaN,Calbryte-520,NaN,7.4,8,20x


In [21]:
pd.to_datetime(protocols["date"], infer_datetime_format=True, errors="coerce")

0     NaT
1     NaT
2     NaT
3     NaT
4     NaT
       ..
537   NaT
538   NaT
539   NaT
540   NaT
541   NaT
Name: date, Length: 542, dtype: datetime64[ns]

In [7]:
protocols["Slice number"].unique()

array([ 1.,  2.,  3., nan,  4.,  5.,  6.,  7.,  8.,  9., 11., 10., 16.,
       15., 14., 13., 12.])

In [2]:
folder = "local_data/prototype/"

In [5]:
for f in sorted(os.listdir(folder)):
    if f.endswith(".meta"): continue
    print (f)
    rec = Recording(os.path.join(folder,f), metadata = "original")
    rec.save_metadata()

Experiment48b.lif
Experiment48d.lif
Experiment49a.lif
Experiment49b.lif
Experiment49c.lif
Experiment49d.lif
Experiment49e.lif
Experiment49f.lif
Experiment50a.lif
Experiment50b.lif
Experiment51a.lif
Experiment51b.lif
Experiment52a.lif
Experiment52b.lif
Experiment52c.lif
ExperimentOGB.lif


In [6]:
javabridge.kill_vm()